In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import os, glob
from IPython.display import display, clear_output


In [ ]:
class bids_input():
    def __init__(self,
                 bids_path = "e.g. /projects/niblab/bids_projects/Experiments/bro/bids",
                 heuristic_file = "e.g. /project/niblab/bids_projects/Experiments/bro/bids/code/bro_heuristic.py",
                 session = "", subjects="e.g. sub-045, sub-050"):
        
        ## Initialize widgets
        out = widgets.Output()
        button_run = widgets.Button(description="RUN")
        # setup text boxes
        style = {'description_width': 'initial'}

        self.bids_path = widgets.Text(description = "Bids Path: ", value= bids_path, style=style)
        self.heuristic_file = widgets.Text(description = "Heuristic File Path: ", value= heuristic_file, style=style)
        self.session = widgets.Text(description = "session: ", value= session, style=style)
        self.subjects = widgets.Text(description = "Enter subject(s): ", value= session, style=style)
        
        # setup buttons
        #button_input = widgets.Button(description="ENTER INPUT", tooltip="Click me")
        
        # setup dropdown menus
        self.sess_dropdown = widgets.Dropdown(
            options=["Select", 'Yes', 'No'],
            value='Select',
            description='Multi-Session:',
            style=style
            )
        
        self.sub_dropdown = widgets.Dropdown(
            options=['Select', 'single/specific ids', 'all'],
            value='Select',
            description='Input subjects:', style=style
            )
        
        
        ## Functions 
       
        # Function for when the session case is selected
        
        def sess_on_change(change):
            if change['type'] == 'change' and change['name'] == 'value':
                if change['new'] == "Yes":
                    display(self.session)#, button_sess)
                    
                else:
                    self.session.close()
                    #button_sess.close()
                    self.session = widgets.Text(description = "session: ", value= session, style=style)

                    
                    
        # Function for when the subject input is selected
        
        def sub_on_change(change):
            if change['type'] == 'change' and change['name'] == 'value':
                if change['new'] == "all":
                    try:
                        self.subjects.close()
                        self.subjects = widgets.Text(description = "Enter subject(s): ", value= session, style=style)
                        self.subjects.value = 'all'
                        
                    except:
                        pass
                    
                elif change['new'] == "single/specific ids":   
                    self.subjects = widgets.Text(description = "Enter subject(s): ", value= session, style=style)
                    display(self.subjects)

                    
                else:
                    pass
                
            
                
        def print_bids():
            heuristic = self.heuristic_file.value
            subjects = self.subjects.value
            session_id = self.session.value
            output = self.bids_path.value
            
            if self.sess_dropdown.value == "Yes":
                
                # setup dicom path
                dicom_path = os.path.join(self.bids_path.value, "sourcedata/DICOM", "sub-{subject}/*")
                
                # case - get all subjects 
                if bids.sub_dropdown.value == "all":
                    
                    subject_lst = [x.split("/")[-1].split("-")[1] for x in glob.glob(os.path.join(self.bids_path.value, "sourcedata/DICOM", "sub-*"))]
                    subject_str = (" ").join(subject_lst)
                    
                else:
                    subject_str= subjects
                    
                singularity_cmd = "singularity exec -B /:/base_dir /projects/niblab/bids_projects/Singularity_Containers/heudiconv_05_2019.simg heudiconv -b -d {} -s {} -ss {} -f {} -c dcm2niix -o {}".format(dicom_path, subject_str, session_id , heuristic, output)
                
                print("SINGULARITY CMD: ", singularity_cmd)

                
            # No sessions (sinlge study) case
            
            elif self.sess_dropdown.value == "No":
                
                # setup dicom path
                dicom_path = os.path.join(self.bids_path.value, "sourcedata/DICOM", "sub-{subject}/*")
                
                
                # case - get all subjects 
                
                if self.sub_dropdown.value == "all":
                    subject_lst = [x.split("/")[-1].split("-")[1] for x in glob.glob(os.path.join(bids.bids_path.value, "sourcedata/DICOM", "sub-*"))]
                    subject_str = (" ").join(subject_lst)
                    #print("SUBJECTS: ",subject_str)
                    
                # specific subjects(s)   
                
                else:
                    

                    subject_str = subjects


                singularity_cmd = "singularity exec -B /:/base_dir /projects/niblab/bids_projects/Singularity_Containers/heudiconv_05_2019.simg heudiconv -b -d {} -s {} -f {} -c dcm2niix -o {}".format(dicom_path, subject_str, heuristic, output)
                
                print("SINGULARITY CMD: ", singularity_cmd)
    

            
            
        def print_output():
            
            if not self.session.value:
                session = "None"
            else:
                session = self.session.value
                
            if self.sub_dropdown.value == "Select":
                print("ERROR: select subject")
                
            elif self.sess_dropdown.value == "Select":
                print("ERROR: select session")
                
            else:    
                if self.subjects.value == 'all':
                    print("bids path: {} \nheuristic file: {} \nsession: {} \nsubjects: {}".format(self.bids_path.value,
                                                                                          self.heuristic_file.value,
                                                                                          session,
                                                                                          self.subjects.value))

                else:
                    if not self.subjects.value:
                        print("ERROR: enter subject id(s)")
                    else:                
                        print("bids path: {} \nheuristic file: {} \nsession: {} \nsubjects: {}".format(self.bids_path.value,
                                                                                          self.heuristic_file.value,
                                                                                          session,
                                                                                        self.subjects.value))
                        print_bids()
                       
                    

        # button click function
            
        def on_input_button_clicked(b):
            with out:
                clear_output()
                print_output()


           
        # class submit function
        def handle_submit(self, text):
            self.v = text.value
            
            return self.v
        
        
        ## Display widgets
        
        display(self.bids_path, self.heuristic_file, self.sess_dropdown, self.sub_dropdown)#, button_input)


        ## Cases for widget interactions 
        
        # cases for when dropdown menus selections are changed
        self.sess_dropdown.observe(sess_on_change)
        
        self.sub_dropdown.observe(sub_on_change)
        
        # when input button is clicked
        #button_input.on_click(on_input_button_clicked)
    
        display(out)
        
        button_input = widgets.Button(description="VIEW INPUT")
        display(button_input)
        button_input.on_click(on_input_button_clicked)
            
        
    



bids = bids_input()


#print("Date input captured: " + "/".join([f.bids_path.value, f.heuristic_file.value]))
